In [1]:
import requests
import os
import yaml
os.chdir('../')

In [41]:
import pandas as pd

In [8]:
from cloudant.view import View
from cloudant.client import Cloudant
from cloudant.document import Document

In [3]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [6]:
cfg = cfg['COUCHDB']
client = Cloudant(cfg['user'], cfg['password'], url=cfg['host'])
client.connect()
db = client[cfg['tweets_db']]

In [17]:
ddoc = Document(db, '_design/sentiment_analysis')

In [55]:
view = View(ddoc, 'geo_sentiment_counts', )

In [58]:
view.items()

dict_items([])

In [50]:
with view.custom_result(group=True, stale='update_after', descending=True) as rslt:
    df = pd.DataFrame.from_records(rslt.all())
df

,key,value
0,"[Positive, Wide Bay, Rest of Qld, Queensland]",15
1,"[Positive, Western Australia - Wheat Belt, Res...",11
2,"[Positive, Western Australia - Outback (South)...",39
3,"[Positive, Western Australia - Outback (North)...",3
4,"[Positive, Warrnambool and South West, Rest of...",30
5,"[Positive, Townsville, Rest of Qld, Queensland]",76
6,"[Positive, Toowoomba, Rest of Qld, Queensland]",17
7,"[Positive, Sydney - Sutherland, Greater Sydney...",8
8,"[Positive, Sydney - South West, Greater Sydney...",1409
9,"[Positive, Sydney - Ryde, Greater Sydney, New ...",4
